In [1]:
#Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Caminho para o modelo salvo
save_path = '/content/drive/My Drive/modelo_treinadoDistilbert'

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install python-telegram-bot==13.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.3.3
    Uninstalling cachetools-5.3.3:
      Successfully uninstalled cachetools-5.3.3


In [3]:
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, CallbackContext, Filters
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [5]:
#Carregar o modelo e o tokenizer salvos
model_name = "Distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(save_path)
model = AutoModelForSequenceClassification.from_pretrained(save_path)

Link para o bot no telegram: t.me/ProdutosTecnologicosBot

In [6]:
#Mapeamento de categorias
label_to_category = {
    0: 'informática',
    1: 'eletrodomésticos',
    2: 'celulares'
}

#Função para classificar produtos
def classify_product(product_name):
    inputs = tokenizer(product_name, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label = torch.argmax(predictions, dim=1).item()
    return label_to_category[predicted_label]

#Token do bot (substitua pelo seu token)
TELEGRAM_TOKEN = '7383300878:AAE9h3AWDbKu0_ZZMUsg7o-CclM79UiCJ20'

#Configurar logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#Função de start
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Olá! Envie o nome de um produto e eu direi a qual categoria ele pertence.')

#Função para responder mensagens
def classify(update: Update, context: CallbackContext) -> None:
    product_name = update.message.text
    category = classify_product(product_name)
    update.message.reply_text(f'O produto "{product_name}" pertence à categoria: {category}')

#Função de ajuda
def help_command(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Envie o nome de um produto e eu direi a qual categoria ele pertence.')

#Função principal
def main() -> None:
    updater = Updater(TELEGRAM_TOKEN, use_context=True)

    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, classify))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()


ERROR:telegram.ext.updater:Error while getting Updates: urllib3 HTTPError ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
ERROR:telegram.ext.dispatcher:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 614, in urlopen
    httplib_response = self._make_request(conn, method, url,
  File "/usr/local/lib/python3.10/dist-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 402, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.10/dist-packages/telegram/vendor/ptb_urllib3/urllib3/connectionpool.py", line 398, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
